<a href="https://colab.research.google.com/github/yashlal/Deepfake-Microbiomes/blob/main/NetGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import torch.optim as optim
import time
from math import sqrt

# select CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if str(device) == 'cuda:0':
	print('CUDA device selected!')
elif str(device) == 'cpu':
	print('CUDA device not available. CPU selected')

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(462, 462*5)
        self.fc2 = nn.Linear(462*5, 462*5)
        self.fc3 = nn.Linear(462*10, 462*10)
        self.fc4 = nn.Linear(462*10, 231*461)
        self.dropout = nn.Dropout(0.1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
       	x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

net = MyNet().to(device)
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=1e-4)

for n in range(1,8):
    d_PATH = f'drive/MyDrive/YashData/{n}'
    f = open(d_PATH, 'rb')
    data = pickle.load(f)

    for i in range(1000):
        optimizer.zero_grad()

        input = torch.from_numpy(data[i][1]).float().to(device)
        true_y = torch.FloatTensor(data[i][0]).to(device)

        output = net(input).to(device)

        loss = criterion(output, true_y).to(device)

        loss.backward()

        optimizer.step()

        print(f'Epoch {i}: {sqrt(loss.item())}')

PATH = 'model.pth'
torch.save(net.state_dict(), PATH)

CUDA device selected!


Output()

Epoch 0: 0.5769283953140336
Epoch 1: 0.5784328517387007
Epoch 2: 0.5760460409052383
Epoch 3: 0.5737825930028958
Epoch 4: 0.5725837057879799
Epoch 5: 0.571478683432908
Epoch 6: 0.5679790102361194
Epoch 8: 0.5601631472706957
Epoch 9: 0.5557072627700402
Epoch 10: 0.5497337824089097
Epoch 11: 0.5420835400824576
Epoch 12: 0.5345351798931441
Epoch 13: 0.5238961160728074
Epoch 14: 0.509817683459306
Epoch 15: 0.4993604056096057
Epoch 16: 0.48215308790352457
Epoch 17: 0.46490325867386134
Epoch 18: 0.44408297202311
Epoch 19: 0.4241719587173292
Epoch 20: 0.3989602194294117
Epoch 21: 0.38121135781616095
Epoch 22: 0.34967965213536223
Epoch 23: 0.3309950535720127
Epoch 24: 0.3059337754456343
Epoch 25: 0.30108729806429935
Epoch 26: 0.3027173068249529
Epoch 27: 0.31387058344050106
Epoch 28: 0.32971604933223353
Epoch 29: 0.32602477040045447
Epoch 30: 0.3360949080982411
Epoch 31: 0.33825885134715694
Epoch 32: 0.3308389461475206
Epoch 33: 0.3229713803771545
Epoch 34: 0.3103103336631245
Epoch 35: 0.300788

KeyboardInterrupt: ignored